## Reading text from file and divid it in label called label and body text from text file

In [9]:
# import nltk
import pandas as pd 
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string 

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv('SMSSpamCollection.tsv', sep='\t')
data.columns = ['label', 'body_text']

def count_punch(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text)-text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punch(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

## Spliting dataset into train / test set 

In [10]:
from sklearn.model_selection import train_test_split

X_train , x_test, Y_train , y_test = train_test_split(data[['body_text', 'body_len', 'punct%']], data['label'], test_size = 0.2)

# Vectoriztion 

In [15]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
tfidf_vect_fit = tdidf_vect.fit(X_train['body_text'])

tfidf_train = tfidf_vect_fit.transform(X_train['body_text'])
tfidf_test = tfidf_vect_fit.transform(x_test['body_text'])

X_train_vect= pd.concat([X_train[['body_len', 'punct%']].reset_index(drop = True), pd.DataFrame(tfidf_train.toarray())], axis = 1)

X_test_vect= pd.concat([x_test[['body_len', 'punct%']].reset_index(drop = True), pd.DataFrame(tfidf_test.toarray())], axis = 1)
X_train_vect.head(5)

,body_len,punct%,0,1,2,3,4,5,6,7,...,7171,7172,7173,7174,7175,7176,7177,7178,7179,7180
0,127,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,22,13.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,63,4.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,60,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,41,2.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Evaluation Model

In [8]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score

In [19]:
rf = RandomForestClassifier(n_estimators= 150, max_depth=None, n_jobs=-1)
rf_model = rf.fit(X_train_vect, Y_train)


y_pred = rf_model.predict(X_test_vect)

precision , recall, fscore , support  = score(y_test, y_pred, pos_label = 'spam', average='binary')

print ('Precision : {} | Recall : {}| Accuracy : {} |'.format( round(precision, 3)
                                                                          , round(recall, 3),
                                                                        round((y_pred==y_test).sum()/len(y_pred), 3)))

Precision : 0.992 | Recall : 0.828| Accuracy : 0.975 |


In [20]:
gb = GradientBoostingClassifier(n_estimators=150, max_depth=11)

gb_model = gb.fit(X_train_vect, Y_train)
y_pred = gb_model.predict(X_test_vect)


precision , recall, fscore , support  = score(y_test, y_pred, pos_label = 'spam', average='binary')

print ('Precision : {} | Recall : {}| Accuracy : {} |'.format( round(precision, 3)
                                                                          , round(recall, 3),
                                                                        round((y_pred==y_test).sum()/len(y_pred), 3)))

Precision : 0.913 | Recall : 0.866| Accuracy : 0.969 |
